<a href="https://colab.research.google.com/github/yirlenarias/TFG_PD/blob/main/Datos/Observado/Lluvia/Lluvia_Observada_20230501.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Climatología de Lluvia Observada

* Para trabajar más cómodamente vamos a descargar todos los ficheros del repositorio GitHub
* De esa manera dispondremos de los datos localmente para nuestro análisis.

In [1]:
# Directorio de trabajo local en la nube de Google Colab 
# ------------------------------------------------------------
%cd /content

# Si existe una copia previa del repositorio, la borramos:
# ----------------------------------------------------------------------------
!  rm -rf TFG_PD

# Creamos una copia del repositorio SOLO si no existe previamente
# ----------------------------------------------------------------------------
! [ ! -d TFG_PD ] && git clone https://github.com/yirlenarias/TFG_PD.git

# Entramos en el repositorio que acabamos de copiar
# --------------------------------------------------
%cd TFG_PD/Datos/Observado/Lluvia

/content
Cloning into 'TFG_PD'...
remote: Enumerating objects: 199, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 199 (delta 27), reused 98 (delta 25), pack-reused 93
Receiving objects: 100% (199/199), 31.78 MiB | 18.36 MiB/s, done.
Resolving deltas: 100% (42/42), done.
/content/TFG_PD/Datos/Observado/Lluvia


## Importar librerias

In [27]:
import glob
import pandas as pd
import os
import zipfile

## Abrir todos los archivos excel por cuenca

In [13]:
# Criterios de coincidencia de patrones para archivos de Excel en el directorio
filenames = 'Cuenca*.xlsx'

In [10]:
filenames

['Cuenca100.xlsx',
 'Cuenca200.xlsx',
 'Cuenca69.xlsx',
 'Cuenca71.xlsx',
 'Cuenca72.xlsx',
 'Cuenca73.xlsx',
 'Cuenca74.xlsx',
 'Cuenca76.xlsx',
 'Cuenca78.xlsx',
 'Cuenca80.xlsx',
 'Cuenca81.xlsx',
 'Cuenca82.xlsx',
 'Cuenca83.xlsx',
 'Cuenca84.xlsx',
 'Cuenca85.xlsx',
 'Cuenca86.xlsx',
 'Cuenca87.xlsx',
 'Cuenca88.xlsx',
 'Cuenca90.xlsx',
 'Cuenca94.xlsx',
 'Cuenca96.xlsx',
 'Cuenca98.xlsx']

In [14]:
# Crea un diccionario vacío para almacenar los datos de cada archivo
dat = {}

# Recorra cada archivo de Excel que coincida con el patrón y lea sus datos en un marco de datos de pandas
for file in sorted(glob.glob(filenames)):
    dat[file] = pd.read_excel(file)

In [15]:
dat

{'Cuenca100.xlsx':        CUENCA  ESTACION      FECHA  LLUVIA_DIARIA
 0         100       633 2004-02-12           0.00
 1         100       633 2004-02-13           1.78
 2         100       633 2004-02-14           0.51
 3         100       633 2004-02-15           0.00
 4         100       633 2004-02-16           0.00
 ...       ...       ...        ...            ...
 29854     100       655 2022-12-27           0.20
 29855     100       655 2022-12-28           0.00
 29856     100       655 2022-12-29           7.80
 29857     100       655 2022-12-30           0.60
 29858     100       655 2022-12-31           0.20
 
 [29859 rows x 4 columns],
 'Cuenca200.xlsx':       CUENCA  ESTACION      FECHA  LLUVIA_DIARIA
 0        200         2 2002-01-01            6.0
 1        200         2 2002-01-02            1.3
 2        200         2 2002-01-03            1.6
 3        200         2 2002-01-04           42.2
 4        200         2 2002-01-05            0.0
 ...      ...       ...

In [16]:
# Access the data from each file using its file name as the key in the data_dict
data_new = []

for file, data in dat.items():
    data_new.append(data)

## Arreglar datos

In [19]:
# Definir una función para aplicar a los datos y en la columna 'ESTACION' todos los valores tengan 3 digitos
for i, df in enumerate(data_new):
    def formatear_valor(valor):
        if valor < 10:
            return '00' + str(valor)
        elif valor < 100:
            return '0' + str(valor)
        else:
            return str(valor)

    # Aplicar la función a la tabla
    data_new[i]['ESTACION'] = data_new[i]['ESTACION'].apply(formatear_valor)

In [20]:
#Unir el numero de 'CUENCA' con 'ESTACION' para tener el numero de cada estacion
for i, df in enumerate(data_new):
    data_new[i]['ESTACION_NEW'] = data_new[i]['CUENCA'].astype(str) + data_new[i]['ESTACION'].astype(str)

In [21]:
#Eliminar las columnas que no son necesarias
for i, df in enumerate(data_new):
    data_new[i].pop('CUENCA')
    data_new[i].pop('ESTACION')

In [22]:
len(data_new)

22

## Agrupar datos por estación

In [23]:
# Agrupar las filas según los valores de la columna "ESTACION_NEW"
my_list = list(range(0, 22))
porestacion=[]
for i in my_list:
    grupos = data_new[i].groupby('ESTACION_NEW')

    # Crear una tabla separada para cada grupo segun 'ESTACION_NEW'
    for ESTACION, grupo in grupos:
        # Obtener el DataFrame correspondiente a cada grupo
        tabla = grupos.get_group(ESTACION)
        porestacion.append(tabla) #Adjuntar los valores

* Obtener el número de cada estación

In [24]:
NUMERO = porestacion.copy()
for i, df in enumerate(NUMERO):
    NUMERO[i] = NUMERO[i].iloc[0,2]

## Guardar un archivo por cada estación

* Crear nombre para cada archivo

In [25]:
ESTACION = []
for i, df in enumerate(ESTACION):
    ESTACION[i] = 'Estacion' + NUMERO[i]

In [26]:
#largo del archivo
len(porestacion)

154

* Guardar cada tabla de cada estación en un CSV diferente

In [28]:
my_list = list(range(0, 154))
with zipfile.ZipFile('tablas.zip', 'w') as mi_zip:
  for a, i in zip(ESTACION,my_list):
      porestacion[i].to_csv(f'{a}.csv', index=True)

## Obtener fecha inicial y final de cada estación

In [ ]:
for i, df in enumerate(porestacion):
    porestacion[i] = porestacion[i].set_index('FECHA')

In [ ]:
# Encontrar los valores de inicio y final de las fechas
my_list = list(range(0, 154))
porestacion_new=[]
for i in my_list:
    fecha_inicial = porestacion[i].index.min()
    fecha_final = porestacion[i].index.max()
    name = NUMERO[i]
    porestacion_new.append((fecha_inicial,fecha_final, name))

* Guardar el rango de fechas en un archivo .txt

In [ ]:
# Abrir el archivo en modo escritura
with open("rango_fechas.txt", "w") as archivo:
    # Escribir la lista en el archivo
    for valor in porestacion_new:
        archivo.write(str(valor) + "\n")

## Obtener los estadisticos

* (media, mediana, máximo y mínimo)

In [ ]:
#Eliminar las columnas que no se necesitan
for i, df in enumerate(data_new):
    porestacion[i].pop('ESTACION_NEW')

In [ ]:
#Obtener el promedio diario
porestacion_mean = []
porestacion_median = []
porestacion_max = []
porestacion_min = []
for i, df in enumerate(porestacion):
    media = porestacion[i].groupby(porestacion[i].index.strftime("%m%d")).mean()
    mediana = porestacion[i].groupby(porestacion[i].index.strftime("%m%d")).median()
    maximo = porestacion[i].groupby(porestacion[i].index.strftime("%m%d")).max()
    minimo = porestacion[i].groupby(porestacion[i].index.strftime("%m%d")).min()
    #groupby() se usa agrupando por mes y por día a la vez
    porestacion_mean.append(media)
    porestacion_median.append(mediana)
    porestacion_max.append(maximo)
    porestacion_min.append(minimo)

* Crear una lista con los estadisticos

In [ ]:
#Crear una lista vacía llamada lista_de_listas_de_tablas
lista_de_listas_de_tablas = []

# Agregar las dos listas de tablas a la lista_de_listas_de_tablas
lista_de_listas_de_tablas.append(porestacion_mean)
lista_de_listas_de_tablas.append(porestacion_median)
lista_de_listas_de_tablas.append(porestacion_max)
lista_de_listas_de_tablas.append(porestacion_min)

* Seleccionar solo la columna que interesa

In [ ]:
# Lista de listas de las columnas seleccionadas
lista_de_columnas_seleccionadas = []

# Iterar a través de cada lista de tablas en lista_de_listas_de_tablas
for lista in lista_de_listas_de_tablas:
    # Crear una lista para almacenar las columnas de 'Nombre' de cada tabla en la lista actual
    columnas_de_nombre = []
    # Iterar a través de cada DataFrame en la lista actual
    for tabla in lista:
        # Seleccionar la columna 'Nombre' de cada tabla y agregarla a la lista de columnas
        columna = tabla['LLUVIA_DIARIA']
        columnas_de_nombre.append(columna)
    # Agregar la lista de columnas de 'Nombre' de la lista actual a la lista de columnas seleccionadas
    lista_de_columnas_seleccionadas.append(columnas_de_nombre)

* Convertir en dataframe cada lista de estaciones

In [ ]:
# Lista de DataFrames individuales
lista_de_dataframes = []

# Iterar sobre cada lista y convertirla en un DataFrame individual
for lista in lista_de_columnas_seleccionadas:
    df = pd.DataFrame(lista)
    lista_de_dataframes.append(df)

* Transponer cada dataframe de estaciones

In [ ]:
# Lista de DataFrames transpuestos
lista_de_dataframes_transpuestos = []

# Iterar sobre cada DataFrame y transponerlo
for df in lista_de_dataframes:
    df_transpuesto = df.transpose()
    lista_de_dataframes_transpuestos.append(df_transpuesto)

* Renombrar las columnas

In [ ]:
# Cambiar los nombres de las columnas en cada DataFrame
for df in lista_de_dataframes_transpuestos:
    df.columns = NUMERO

## Guardar estadisticos en excel

In [ ]:
# Crear el objeto ExcelWriter
writer = pd.ExcelWriter('Climatologia_Diaria_Observada_Lluvia_20230429.xlsx', engine='xlsxwriter')

# Escribir los DataFrames en diferentes hojas
lista_de_dataframes_transpuestos[0].to_excel(writer, sheet_name='mean', index=True)
lista_de_dataframes_transpuestos[1].to_excel(writer, sheet_name='median', index=True)
lista_de_dataframes_transpuestos[2].to_excel(writer, sheet_name='max', index=True)
lista_de_dataframes_transpuestos[3].to_excel(writer, sheet_name='min', index=True)

# Guardar el archivo Excel
writer.save()

## Abrir lista de estaciones

In [ ]:
#Cargar la lista de estaciones (enviadas oor Keily) y que que contienen el rango de fechas que agregamos manualmente del .txt guardado a partir de las fechas de inicio y final
estaciones = pd.read_excel(r'/Users/arias/OneDrive/Documentos/UCR/TFG/Datos/Observada/Lluvia/Lista de estaciones.xlsx')

In [ ]:
estaciones

,NUMERO,NOMBRE,LATITUD NORTE,LONGITUD OESTE,ALTITUD (msnm),INICIO,FINAL,ESTACION
0,100633,FINCA EL PATIO. PUERTO JIMENEZ,"08º 36' 04.6""","83º25'52.2""",24,2004-02-12,2022-12-31,100633
1,100641,COOPEAGROPAL. LAUREL .,"08º 28' 17.6""","82º51'38.4""",38,2007-03-09,2022-12-31,100641
2,100643,RIO CLARO,"08º 40' 26.5""","83º03'43.2""",51,2009-12-11,2022-12-31,100643
3,100649,FUNDACION NEOTROPICA. RINCON DE OSA,"08º 42' 2.9""","83º30'49.4""",80,2013-01-16,2022-11-10,100649
4,100651,COTO 49. CIUDAD NEILY,"08º 37' 52.3""","82º58'00.7""",30,2013-05-16,2022-12-31,100651
...,...,...,...,...,...,...,...,...
149,98095,ALTAMIRA,"09º 01' 45.7""","83º00'28.7""",1370,2002-01-01,2022-12-31,98095
150,98097,MONTECARLO. PZ,"09º 21' 28""","83º36'03""",1005,2008-06-17,2022-12-31,98097
151,98105,PLANTA DE VOLCAN. BUENOS AIRES,"09º 12' 15""","83º27'2.49""",416,2013-11-26,2022-12-07,98105
152,98107,CAFETALERA EL INDIO. SAN VITO. PUNTARENAS,"08º 49' 57.9""","82º57'10.5""",1027,2016-12-15,2022-11-10,98107


In [ ]:
# Convertir las columnas de fecha a objetos datetime
estaciones['INICIO'] = pd.to_datetime(estaciones['INICIO'])
estaciones['FINAL'] = pd.to_datetime(estaciones['FINAL'])

In [ ]:
# Seleccionar filas con 10 o más años de diferencia entre A y B
df_10_anos = estaciones[(estaciones['FINAL'] - estaciones['INICIO']) >= pd.Timedelta('3650 days')]
df_10_anos

,NUMERO,NOMBRE,LATITUD NORTE,LONGITUD OESTE,ALTITUD (msnm),INICIO,FINAL,ESTACION
0,100633,FINCA EL PATIO. PUERTO JIMENEZ,"08º 36' 04.6""","83º25'52.2""",24,2004-02-12,2022-12-31,100633
1,100641,COOPEAGROPAL. LAUREL .,"08º 28' 17.6""","82º51'38.4""",38,2007-03-09,2022-12-31,100641
2,100643,RIO CLARO,"08º 40' 26.5""","83º03'43.2""",51,2009-12-11,2022-12-31,100643
7,200002,ISLA DEL COCO,"05º 33' 00""","87º03'00""",75,2002-01-01,2017-07-11,200002
8,200007,ISLA SAN JOSE,"10º 51' 15.40""","85º54'34.50""",44,2008-05-24,2019-08-15,200007
...,...,...,...,...,...,...,...,...
146,98075,LAS BRISAS. SABALITO,"08º 52' 38.3""","82º53'27.5""",1073,2002-01-01,2022-11-10,98075
147,98087,PINDECO,"09º 08' 49""","83º20'06""",397,2002-01-01,2022-12-31,98087
148,98091,CHIRRIPO,"09º 27' 30""","83º30'30""",3440,2002-01-01,2022-04-18,98091
149,98095,ALTAMIRA,"09º 01' 45.7""","83º00'28.7""",1370,2002-01-01,2022-12-31,98095


In [ ]:
# Calcular la cantidad de años en cada fila y encontrar la máxima
estaciones['diff'] = estaciones['FINAL'].dt.year - estaciones['INICIO'].dt.year
max_diff = estaciones['diff'].max()

# Seleccionar solo las filas que tienen la cantidad máxima de años
df_max_diff = estaciones[estaciones['diff'] == max_diff]

In [ ]:
df_max_diff

,NUMERO,NOMBRE,LATITUD NORTE,LONGITUD OESTE,ALTITUD (msnm),INICIO,FINAL,ESTACION,diff
10,69633,COMANDO LOS CHILES,"11º 01' 54""","84º42'42""",40,2002-01-01,2022-12-31,69633,20
11,69647,FINCA BRASILIA DEL ORO,"10º 58' 58.9""","85º20'50.7""",381,2002-11-07,2022-12-31,69647,20
13,69679,UPALA,"10º 52' 51.5""","85º04'22.1""",60,2002-01-01,2022-12-31,69679,20
14,69681,LA REBUSCA,"10°28'28.3""","83°00'32.8""",40,2002-01-01,2022-12-31,69681,20
35,71015,CANTA GALLO,"10°29'48.9""","83°40'16.7""",30,2002-01-01,2022-12-31,71015,20
39,72157,FINCA LA CEIBA,"10º 06' 40""","85º19'03""",58,2002-01-01,2022-12-31,72157,20
40,72159,PAQUERA.,"09º 49' 17""","84º56'20""",10,2002-10-27,2022-12-31,72159,20
54,73123,ITCR. CARTAGO,"09º 51' 08""","83º54'31""",1360,2002-01-01,2022-12-31,73123,20
55,73129,RECOPE. OCHOMOGO,"09º 53' 40.21""","83º56'19.41""",1546,2002-01-01,2022-06-08,73129,20
72,74051,AEROP. LIBERIA OESTE 07,"10º 35' 20.40""","85º33'07.70""",89,2002-01-01,2022-12-31,74051,20
